In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("eps130_hw2_EQ_Forecasting_v1.1.ipynb")

# Probability of Occurrence of Mainshocks and Aftershocks

## Introduction
In the previous homework we learned about the particularly well-behaved statistics of the earthquake magnitude distribution, and aftershock occurrence. As we saw, it is possible to use the frequency of event occurrence over a range of magnitudes to extrapolate to the less frequent large earthquakes of interest. How far this extrapolation may be extended depends upon a number of factors. It is certainly not unbounded as fault dimension, segmentation, strength and frictional properties will play a role in the maximum size earthquake that a fault will produce. Paleoseismic data is used to provide a better understanding of the recurrence of the large earthquakes of interest. The large earthquakes have greater fault offset, rupture to the surface of the Earth and leave a telltale geologic record. This record is used to determine the recurrence of the large characteristic earthquakes and probabilistic earthquake forecasts.  Finally, this type of analysis is perhaps one of the most visible products of earthquake hazard research in that earthquake forecasts and probabilities of aftershock occurrence are generally released to the public.

## Objective
In this homework we will assume a Poisson distribution to determine the probability of events based on the Gutenberg-Richter recurrence relationship.  Given the statistical aftershock rate model of Reasenberg and Jones (1996) we will forecast the probability of occurrence of large aftershocks for the 2014 Napa earthquake sequence. For the Mojave segment of the San Andreas Fault we will compare probability density models to the recurrence data and use the best fitting model to determine the 30-year conditionally probability of occurrence of a magnitude 8 earthquake. 

Use the code provided in this Jupyter Notebook to analyze the provided data, and then answer the questions to complete this homework. Submit your completed notebook in a \*.pdf format. Write your answers embedded as Markdown inside the notebook where specified.

In [1]:
#Initial Setup and Subroutine Definitions
import math
import datetime
import numpy as np
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
def countDays(c,y,m,d):
    days=np.zeros(c)
    for i in range(0,c,1):
        d0 = datetime.date(y[0], m[0], d[0])
        d1 = datetime.date(y[i], m[i], d[i])
        delta = d1 - d0
        days[i]=delta.days
    return days
def readAnssCatalog(p):
    # slices up an ANSS catalog loaded as a pandas dataframe and returns event info
    d=np.array(p)         # load the dataframe into numpy as an array      
    year=d[:,0].astype(int)  # define variables from the array
    month=d[:,1].astype(int)
    day=d[:,2].astype(int)
    hour=d[:,3].astype(int)
    minute=d[:,4].astype(int)
    sec=d[:,5].astype(int)
    lat=d[:,6]
    lon=d[:,7]
    mag=d[:,8]
    days = countDays(len(year),year,month,day)
    return year,month,day,hour,minute,sec,lat,lon,mag,days

## Exercise 1

The simplest model description of the probability that an earthquake of a given magnitude will happen is that of random occurrence. In fact when you examine the earthquake catalog it does in fact appear to be randomly distributed in time with the exception of aftershocks and a slight tendency of clustering. The Poisson distribution is often used to examine the probability of occurrence of an event within a given time window based on the catalog statistics. A Poisson process occurs randomly with no “memory” of time, size or location of any preceding event. Note that this assumption is inconsistent with the implications of elastic rebound theory applied to a single fault for large repeating earthquakes, but is consistent with the gross seismicity catalog.  

The Poisson distribution is defined as,

$$
p(x)=\frac{u^x e^{-u}}{x!}
$$

where $x$ is the number of events, and $u$ is the number of events that occur in time $\delta t$ given the rate of event occurrence $\lambda$, or $u = \lambda*\delta t$. Consider the case in which we would like to know the probability of an event of a certain magnitude occurring within a certain time. Using the Poisson distribution, we can define the probability of one or more events occuring to be,

$$
p(x >= 1)=1.0 - e^{-u}.
$$

The probability of one or more events occuring in a specified time period, for example $\delta t =$ 30 years, can be shown to be

$$
p(x >= 1)=1.0 - e^{-\lambda \delta t},
$$

where $\lambda$ is the annual rate of event occurrence (N), taken from Gutenberg-Richter analysis.


### Question 1.1

Using the Poisson model estimate the probability of a magnitude 5+ earthquake in a given week, month, year and 5 year period using the annual rate determined from the Gutenberg-Richter relationship for the Greater San Francisco Bay Area:

$$
Log(N) = 3.45 - 0.830M
$$

<!--
BEGIN QUESTION
name: q1.1
-->

In [57]:
# You can use this cell for questions 1 & 2, just skip ahead to the
#    question 2 test cell when using the question 2 magnitude

# Average annual rate of occurrence of M mag+ from G-R stats
mag = ...
lam = ...

# Time range in years
# You can compute all four (week, month, 1y, 5y) probabilities
#    at once by making a list of time ranges in years
dt = [ , , , ] # the tester expects these in ascending order [w,m,1y,5y]

# The probability of an event of Magnitude M occurring
P = [... for t in dt] # this is called a "list comprehension"
# the tester expects plain probability values in the range [0,1]; don't convert to percentages

print('The probability of an M5+ event occurring in 1 week, 1 month, 1 year, and 5 years=')
print(P)

In [ ]:
grader.check("q1.1")

_Type your answer here, replacing this text._

### Question 1.2

Compare the estimated probability of a magnitude 7.0+ earthquake for the same time periods.

<!--
BEGIN QUESTION
name: q1.2
-->

In [ ]:
grader.check("q1.2")

_Type your answer here, replacing this text._

## Exercise 2

The Poisson probability function above may also be used to determine the probability of one or more aftershocks of given magnitude range and time period following the mainshock. 

Typically an estimate of the probability of magnitude 5 and larger earthquakes is given for the period of 7 days following a large mainshock. This aftershock probability estimate is found to decay rapidly with increasing time.  Reasenberg and Jones (1989) studied the statistics of aftershocks throughout California and arrived at the following equation describing the rate of occurrence of one or more events as a function of elapsed time for a generic California earthquake sequence:

$$
rate(t,M)=10^{(-1.67 + 0.91*(Mm - M))}   * (t + 0.05)^{-1.08}, 
$$

where Mm is the mainshock magnitude, M is magnitude of aftershocks (can be larger than Mm), and t is time in units of days. This equation describes the daily production rate of aftershocks with magnitude (M) after the mainshock with magnitude Mm. The rate is a function of time (t) and the aftershock magnitude. Elements of both the Gutenberg-Richter relationship and Omori’s Law are evident in the above equation.  

The Poisson probability of one or more aftershocks with magnitude M in range of M1 < M < M2, and time t in range t1 < t < t2 is:

$$
p(M1,M2,t1,t2) = 1.0 - e^{-\int_{M1}^{M2} \int_{t1}^{t2}rate(t,M)dtdM}
$$

The double integral in the exponent may be approximated by nested summations. That is, for each magnitude from M1 to M2 the sum of the rate function over the time period of interest (typically from t1=0 to t2=7 days) can be computed.

We can also evaluate the integral exactly for the number of earthquakes 
in the magnitude range [M1,M2] and time range [t1, t2]:

$$
p(M1,M2,t1,t2) = 1.0 - e^{-\int_{M1}^{M2} \int_{t1}^{t2}rate(t,M)dtdM}
$$
$$
u = \int_{M1}^{M2} \int_{t1}^{t2}rate(t,M)dtdM
$$
$$
u= \int_{M1}^{M2} \int_{t1}^{t2} 10^{(-1.67 + 0.91*(Mm - M))}   * (t + 0.05)^{-1.08} dtdM
$$
$$
u= 10^{-1.67+0.91Mm}  \int_{M1}^{M2} 10^{-0.91M}  dM \int_{t1}^{t2} (t + 0.05)^{-1.08} dt
$$
$$
u= \frac{10^{-1.67+0.91Mm}}{ln(10)(-0.91) (-0.08)}  [10^{-0.91M_2} - 10^{-0.91M_1}][ (t_2 + 0.05)^{-0.08} - (t_1 + 0.05)^{-0.08}]
$$

Then the probability (p(x)) of having one of more earthquakes in the magnitude range [M1,M2] and time range [t1,t2] is:

$$
p = 1-e^{-u}
$$

### Question 2.1
Use these relationships to estimate the probability of one or more magnitude 5 and larger (potentially damaging) aftershocks in the 7 days following the October 18, 1989 Loma Prieta Earthquake studied in Homework 1.

<!--
BEGIN QUESTION
name: q2.1
-->

In [6]:
# For the Loma Prieta earthquake, Mm = 6.9, 
# M1 = 5.0 and M_2 = 6.8 (since the question asks for aftershocks, 
# the aftershock maximum magnitude should be less than the mainshock 
# magnitude, otherwise it will be mainshock and Loma Prieta earthquake will 
# be termed "foreshock".)
#

P = ...

In [ ]:
grader.check("q2.1")

_Type your answer here, replacing this text._

### Question 2.2
By the end of day two how much has the probability of occurrence of a magnitude 5+ aftershock decayed? That is, what is the new 7-day probability starting on day 2?

<!--
BEGIN QUESTION
name: q2.2
-->

In [8]:
P = ...

In [ ]:
grader.check("q2.2")

_Type your answer here, replacing this text._

### Question 2.3
We want to compare the expected number of aftershocks per day for various magnitude thresholds (M > 2, M > 3 etc) and the observed outcome for the Loma Prieta earthquake sequence. Start by making a table of the observed aftershocks per day.

In [10]:
# Load the catalog from HW #1 (provided in your current working directory)
print('The magnitude-time distribution of Loma Prieta aftershocks is shown here:')
data=pd.read_csv('anss_catalog_1900to2018all.txt', sep=' ', delimiter=None, header=None,
                names = ['Year','Month','Day','Hour','Min','Sec','Lat','Lon','Mag'])
EQ_1989 = data[(data.Year>=1989) & (data.Year<1990)]          #get one year of data
fall_eq = EQ_1989[(EQ_1989.Month>9) & (EQ_1989.Month<=12)]    #collect months of Oct, Nov and Dec
LP_eq = fall_eq[(~((fall_eq.Month==10) & (fall_eq.Day<18)))]  #negate events before day (assumes first month is 10)
LP_eq = LP_eq[(~((LP_eq.Month==12) & (LP_eq.Day>18)))]        #negate events after day (assumes last month is 12)
LP_eq.reset_index(drop=True, inplace=True)
year,month,day,hour,minute,sec,lat,lon,mag,days = readAnssCatalog(LP_eq)
days = days[1:] # remove mainshock
mag = mag[1:]

# Plot of magnitude vs. day for entire catalog
fig, ax = plt.subplots(figsize=(7,3))
ax.plot(days,mag,'o',alpha=0.2,markersize=5)
ax.set(xlabel='Days', ylabel='Magnitude',
       title='Raw Event Catalog')
ax.grid()
ax.set_ylim([0,7])
plt.show()

In [11]:
# Count aftershocks each day from 10/18 to 10/25 and make a table aftershocks_observed
aftershock_days = np.arange(18,26) # day dates
aftershock_mags = np.arange(2,6) # mags to count
aftershocks_observed = pd.DataFrame(columns = [f'10/{d}' for d in aftershock_days],
                                    index=[f'M>={m}' for m in aftershock_mags]) # set up table

# Fill in the table with the number of aftershocks per day
# Hint: the easiest way to find the number of aftershocks per day in a magnitude range is to 
# further refine the LP_eq catalog uisng boolean statements.

In [12]:
aftershocks_observed

In [ ]:
grader.check("q2.3")

### Question 2.4
We want to compare the expected number of aftershocks per day for various magnitude thresholds (M > 2, M > 3 etc) and the observed outcome for the Loma Prieta earthquake sequence. Now compute the expected number of aftershocks per day from the analytical integral of the rate function.

In [51]:
Mm = 6.9



aftershocks_RJ = pd.DataFrame(columns = [f'10/{d}' for d in aftershock_days],
                              index=[f'M>={m}' for m in aftershock_mags]) # set up rate table


def RJ(Mm,M1,M2,t1,t2):
    u = ...
    return int(np.round(u,0))

# fill in aftershocks_RJ table
...

In [52]:
aftershocks_RJ

In [17]:
aftershocks_observed

_Type your answer here, replacing this text._

In [ ]:
grader.check("q2.4")

### Question 2.5
The statistics compiled by Reasenberg and Jones also allows for the estimation of the probability of an event larger than the mainshock occurring, or in other words the probability that a given event is in fact a foreshock. Immediately following the Loma Prieta earthquake, after a lapse time of 0.1 day, what was the 7-day probability that a larger earthquake might occur?

<!--
BEGIN QUESTION
name: q2.5
-->

In [18]:
P = ...
print('After 0.1 days, the probability that the Loma Prieta M6.9 earthquake was a foreshock to a larger earthquake was')
print(str(round(P,4)))

In [ ]:
grader.check("q2.5")

<!-- BEGIN QUESTION -->

### Question 2.6
Practically speaking, what was the duration of the Loma Prieta sequence? Explain you answer in terms Omori statistics and the probability of aftershock occurrence with increasing time following the main shock. This is an open-ended question. You might compare pre-event and omori-decay seismicity rates. You could use Reasenberg and Jones to find a time when the probability of a felt earthquake has fallen to a low level.

<!--
BEGIN QUESTION
name: q2.6
manual: true
-->

In [30]:
#You can answer this by 1) comparing pre-event and omori decay rates, 2) and from reasonberg and jones finding the
#time when the probability of say a M3+ earthquake falls to a low level. i.e. integrate from say t1 to t2=infinity

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Exercise 3

As discussed in class paleoseismic trench data at Pallet Creek in southern California reveals the quasi-periodic behavior of large earthquakes on the San Andreas fault. From the very careful mapping of offset stratigraphy in the trench and carbon-14 radiometric dating these large earthquakes have been found to have occurred in 1857, 1812, 1480, 1346, 1100, 1048, 997, 797, 734, 671, 529 (see figure from Sieh, K., Stuiver, M. and Brillinger, D., 1989). These earthquakes include M8 earthquakes on the southern segment of the San Andreas fault, which extends from Parkfield southward through the Big Bend into southern California. Each earthquake may not have been as large as M8, however, given the mapped slip, each event is considered to be M>7. The 1857 earthquake was M8. 

<img src="palletCreek.png">

Using this recurrence data we are going to examine the periodicity, plot the distribution of events in terms of binned interval time, compare the observed distribution with idealized probability density functions, and then use those functions to estimate the conditional probability of occurrence of these events.

<!-- BEGIN QUESTION -->

### Question 3.1

Given the time intervals separating the event list given above, compare the fits of a Gaussian and Lognormal probability density model. 

Gaussian:  
$$
pd(u)=\frac{e^{\frac{-(u - T_{ave})^2}{2 {\sigma}^2}}}{\sigma \sqrt{2 \pi}}
$$


Log-Normal: 
$$
pd(u)=\frac{e^{\frac{-{(ln(u/T_{ave}))}^2}{2 {(\sigma / T_{ave})}^2}}}{(\frac{\sigma}{T_{ave}}) u  \sqrt{2 \pi}}
$$

The models depend on the mean interval recurrence time ($T_{ave}$), the standard deviation to the mean ($\sigma$), and the random variable ($u$) which in this case represents the interval time.


To do this make a histogram with bins from 1-51, 51-101, 101-151, etc. The center dates of the bins will be 26, 76, 126, etc. Then fit each probability density model. This part is done for you.

**Question**: Which type of distribution appears to fit the data better?

<!--
BEGIN QUESTION
name: q3.1
manual: true
-->

In [21]:
# hint: matplotlib.pyplot and pandas.DataFrame both have 
#       histogram functions

# Enter the event years and intervals into a table
# There are other (better) ways to do this, can you think of one?
print('\nInterval Times:')
c = {0:[1857,45],1:[1812,332],2:[1480,134],3:[1346,246],4:[1100,52],5:[1048,51],6:[997,200],7:[797,63],
     8:[734,63],9:[671,142],10:[529,0]}
df = pd.DataFrame.from_dict(data=c,orient='index',columns=['Date','Intervals'])
print(df)

# With so few data points we can get away with manually counting the bins
# Think about how you could make python do more of the work here
print('\nHistogram Data:')
c = {0:['0<=T<51',1],1:['51<=T<101',4],2:['101<=T<151',2],3:['151<T<201',1],
     4:['201<T<251',1],5:['251<T<301',0],6:['301<T<351',1]}
hf = pd.DataFrame.from_dict(data=c,orient='index',columns=['Time Range','Count'])
print(hf)

# Models
Tave = np.mean(df.Intervals) # mean of each bin
sig =  np.std(df.Intervals)
u=np.arange(0.1,351,1,) # number of years spanned by all bins
# Gaussian
uG=np.exp(-(u - Tave)**2/(2*sig**2))/(sig*np.sqrt(2*np.pi))
# Log-normal
uLN=np.exp(-(np.log(u /Tave))**2/(2*(sig/Tave)**2))/((sig/Tave)*u*np.sqrt(2*np.pi))

# Plot the result
plt.figure()
hf.Count.plot(kind='bar');
plt.plot(u*(6/351),uG*500,'r-'); # scaling u and uG to match bar plot dimensions
plt.plot(u*(6/351),uLN*500,'b-');
plt.xticks(range(len(hf)), hf['Time Range'].values, size='small',rotation=30);

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Exercise 4
In this problem we will estimate the probability of occurrence of a magnitude M8 earthquake based on the historic Pallet Creek recurrence data and the best fitting probability density model determined in Exercise 3.  

The probability that an event will occur within a given time window, for example 30-years, is the definite integral of the probability density function computed over that time window: 
$$
P(T_e <= T <= T_e + \Delta T)=\int_{T_e}^{T_e + \Delta T} pd(u)du,
$$

where $\Delta T$ is the length of the forecast window and $T_e$ is the time since the previous event. Note how P varies as a function of elapsed time. For any given forecast window, the value of P is small but is greatest near the mean of the distribution. Note that the Gaussian and lognormal probability density functions defined above are normalized to unit area. 

### Question 4.1
Estimate the 10-year, 20-year and 30-year probabilities for a repeat of this large Pallet Creek fault segment event using your estimates of $T_{ave}$, $\sigma$, and $T_e=164$ years (time since 1857).

The first step is to find the probability that the event will occur in the window, $\Delta T$, with the condition that the event did not occur before $T_e$. This effectively reduces the sample space. The result is the following normalization for the conditional probability:

$$
P(T_e <= T <= T_e + \Delta T | T >= T_e) = \frac{\int_{T_e}^{T_e + \Delta T} pd(u)du}{1.0 - \int_{0}^{T_e}pd(u)du}
$$

In [47]:
Te = ...
Tave = ...
sig = ...

# suggestion: make functions to calculate uG and uLN that you can use again in later questions
def calc_uG(Tave,sig,u):
    uG= ...
    return uG
def calc_uLN(Tave,sig,u):
    uLN= ...
    return uLN

u= ...
uG = calc_uG(Tave,sig,u)
uLN = ...

# if we use a step size of 1 (year) then we can numerically integrate by just taking the sum
pG10_Te = np.sum(uG[Te:(Te+10)])/(1-np.sum(uG[0:Te]))
pLN10_Te = ...
pG20_Te = ...
pLN20_Te = ...
pG30_Te = ...
pLN30_Te = ...
print("Gaussian model")
print("{:.6f}, {:.6f}, {:.6f}".format(pG10_Te,pG20_Te,pG30_Te))
print("Log-normal model")
print("{:.6f}, {:.6f}, {:.6f}".format(pLN10_Te,pLN20_Te,pLN30_Te))

These are the conditional probabilities of an earthquake occurring within a time interval of $\Delta T$ years between $T_e$ And $T_e$+$\Delta T$ years given that it did not occur before time $T_e$ (for $\Delta T$ = 10 years, 20 years, and 30 years).

<!--
BEGIN QUESTION
name: q4.1
-->

In [ ]:
grader.check("q4.1")

<!-- BEGIN QUESTION -->

### Question 4.2

Make two plots showing (a) both pd(u) models for u = [0,500] years and (b) the 10-year, 20-year and 30-year probability windows for $T_e = [0,500]$ years (done for you). Describe the second plot. What does it tell you?

<!--
BEGIN QUESTION
name: q4.2
manual: true
-->

In [49]:
# Plot Models over 500 years
plt.figure()
plt.plot(u,uG,label='Gaussian')
plt.plot(u,uLN,label='Log-Normal')
plt.xlim([0,500])
plt.ylim([0,max(uLN)])
plt.legend()
plt.xlabel('Interval time [years]')
plt.ylabel('Number of Events')

# We can integrate the definite integrals described above, using 
# Gaussian and Log-Normal distributions for pd(u), by np.trapz, np.sum, etc.
te = range(0,5000,1)
pG10 = np.zeros(np.shape(te))
pLN10 = np.zeros(np.shape(te))
for t in te:
    uG = calc_uG(Tave,sig,u)
#     print(np.shape(uG))
    pG10[t] = np.sum(uG[t:t+10])
    uLN = calc_uLN(Tave,sig,u)
    pLN10[t] = np.sum(uLN[t:t+10])

pG20 = np.zeros(np.shape(te))
pLN20 = np.zeros(np.shape(te))
for t in te:
    uG = calc_uG(Tave,sig,u)
    pG20[t] = np.sum(uG[t:t+20])
    uLN = calc_uLN(Tave,sig,u)
    pLN20[t] = np.sum(uLN[t:t+20])
    
pG30 = np.zeros(np.shape(te))
pLN30 = np.zeros(np.shape(te))
for t in te:
    uG = calc_uG(Tave,sig,u)
    pG30[t] = np.sum(uG[t:t+30])
    uLN = calc_uLN(Tave,sig,u)
    pLN30[t] = np.sum(uLN[t:t+30])
    
# Plot Probabilities
plt.figure()
plt.plot(u,pG10,'-',color='r',label='10-year Gaussian');
plt.plot(u,pLN10,'-',color='b',label='10-year Log-Normal');
plt.plot(u,pG20,'--',color='r',label='20-year Gaussian');
plt.plot(u,pLN20,'--',color='b',label='20-year Log-Normal');
plt.plot(u,pG30,':',color='r',label='30-year Gaussian');
plt.plot(u,pLN30,':',color='b',label='30-year Log-Normal');
plt.vlines(x=Tave,ymin=0,ymax=(max(pLN30)),linestyles='-',label='$T_{ave}$')

plt.xlim([0,500])
plt.ylim([0,max(pLN30)])
plt.xlabel('Te [years]');
plt.ylabel('Probability');
plt.legend();

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### Question 4.3

Estimate the change in the 30-year probability if the event does not occur in the next 10 years.

<!--
BEGIN QUESTION
name: q4.3
-->

In [51]:
Te = ...
pLN30_Te = ...
print(f'{pLN30_Te:.6f}')

In [ ]:
grader.check("q4.3")

_Type your answer here, replacing this text._

<!-- BEGIN QUESTION -->

### Question 4.4

Can you identify a weakness of this model?

<!--
BEGIN QUESTION
name: q4.4
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a pdf file for you to submit. **Please save before exporting!** The exporter will not see any unsaved changes to your notebook.

In [ ]:
!../eps130_export eps130_hw2_EQ_Forecasting_v1.0.ipynb

[Access your pdf here.](./eps130_hw2_EQ_Forecasting_v1.0.pdf)

Remember to check that you pdf shows your most recent work before submitting.